In [1]:
import os
from src.dataPipeline import DataPipeline
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10)

os.chdir('..')

dp = DataPipeline()
df = dp.runPipeline(normalizeAndStandardize=False)

C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:39: DtypeWarning: Columns (3,4,5,6,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50,107,110,114,115,116,119,120,121,124,125,126,128,131,132) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(filePath)
C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:62: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ]].bfill(axis=1)['Space extracted']
C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:67: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To o

In [2]:
from src import torchModelRun
import torch
import torch.nn as nn
import torch.nn.functional as F

inputs_nums = len(df.columns) - 1

class FullyConnectedModel(nn.Module):
    import torch.nn.functional as F
    def __init__(self):
        super(FullyConnectedModel, self).__init__()
        self.fc1 = nn.Linear(inputs_nums, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 8)
        self.fc4 = nn.Linear(8, 1)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
    

models = torchModelRun.run(FullyConnectedModel, df, 4)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: denis-schatzmann. Use `wandb login --relogin` to force relogin


Epoch 1, Loss: 133.64595178845286, Train MAPE: nan, Test MAPE: nan
Epoch 2, Loss: 16.344204555571764, Train MAPE: 208.74148513969334, Test MAPE: nan
Epoch 3, Loss: 1.3763191432959732, Train MAPE: 81.5778908060885, Test MAPE: 80.70056893295002
Epoch 4, Loss: 0.6457628226742662, Train MAPE: 66.52171879910874, Test MAPE: 65.50716505485309
Epoch 5, Loss: 0.5498119457100314, Train MAPE: 61.99466845194499, Test MAPE: 61.47004110116422
Epoch 6, Loss: 0.5111943863846105, Train MAPE: 64.14636256557772, Test MAPE: 64.22668991395679
Epoch 7, Loss: 0.4835142846080078, Train MAPE: 58.19843794504801, Test MAPE: 58.39729451749663
Epoch 8, Loss: 0.46321816838335717, Train MAPE: 56.204012947520994, Test MAPE: 56.82634499884802
Epoch 9, Loss: 0.44723313335200837, Train MAPE: 53.182469552138755, Test MAPE: 53.72802733352293
Epoch 10, Loss: 0.43462639478431353, Train MAPE: 55.71729372835707, Test MAPE: 56.90244307530789
Epoch 11, Loss: 0.4226765029132366, Train MAPE: 51.65244998493414, Test MAPE: 52.54196

KeyboardInterrupt: 

In [ ]:
 index_nr = 64
df_exclude = df.drop(columns=['price_cleaned'])
datapoint = df_exclude.iloc[index_nr].values
print(datapoint)
datapoint = torch.tensor(datapoint, dtype=torch.float32).unsqueeze(0)
transformer = models[0]['transformer']

datapoint = transformer(datapoint)


model = models[0]['model']
model.eval()
model = model.to('cpu')
output = model(datapoint)
print(f'output: {output}')
transformer = models[0]['transformer']
print(f'min: {transformer.}, max: {transformer.}')
output = transformer.inverse(output)
print(f'output inverst: {output}')
output = torch.exp(output)
print(f'output exp: {output[0][0]:.0f}')

In [ ]:
df['price_cleaned'].iloc[index_nr]

In [ ]:
df.head()

In [ ]:
train_loader, val_loader, test_loader = torchModelRun.getDataLoaders(df, 'price_cleaned', 1)

counter = 5
for batch in train_loader:
    print(batch)
    counter -= 1
    if counter == 0:
        break

In [ ]:
df.std().values.reshape(1, -1)

In [ ]:
df.columns

In [ ]:
import numpy as np
input = 1150000.0
log = np.log(input+1)
min = 0.6931471824645996
max = 17.70733070373535
output = (log - min) / (max - min)

print(f'output: {output}')

output = output * (max - min) + min
output = np.exp(output) - 1
print(f'output: {output}')


